In [35]:
import copy
import json 
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize 
from io import StringIO
import math
import time

url_all_matches_TI19 = 'https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(matches.leagueid%20%3D%2010749)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272018-08-06T22%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%2010000'
url_base_match = 'https://api.opendota.com/api/matches/'

class DotaTeam:
    def __init__(self,name,wins,losses,winrate,skill,real,luck):
        self.name = name
        self.wins = wins
        self.losses = losses
        self.winrate = winrate
        self.skill = skill
        self.real = real
        self.luck = luck
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)


class DotaMatch:
    def __init__(self,match_id,radiant_score,dire_score,duration,first_blood_time,picks_bans,radiant_team,dire_team):
        self.match_id = match_id
        self.radiant_score = radiant_score
        self.dire_score = dire_score
        self.duration = duration
        self.first_blood_time = first_blood_time
        self.pick_bans = picks_bans
        self.radiant_team = radiant_team
        self.dire_team = dire_team
    def __str__(self):
        return str(self.__dict__)
    def __repr__(self):
        return str(self)

In [3]:
# r = requests.get(url)
r_all_matches_TI19 = requests.get(url_all_matches_TI19)

In [4]:
parsed_json_all_matches_TI19 = json.loads(r_all_matches_TI19.text)
df_all_matches_TI19 = pd.json_normalize(parsed_json_all_matches_TI19)


list_all_matches_TI19 = []

In [8]:
for match in parsed_json_all_matches_TI19['rows']:
    if match['match_id'] not in list_all_matches_TI19:
        list_all_matches_TI19.append(match['match_id'])
print('Number of Matches: ', len(list_all_matches_TI19))

Number of Matches:  461


In [9]:
small_dataset = list_all_matches_TI19 #[:10]

In [12]:
fetch_counter = 1
matchdata_TI19 = []
for match in small_dataset:
    # r = requests.get(url)
    matchdata_response = requests.get(url_base_match + str(match))
    if matchdata_response.status_code != 200:
        raise TypeError("Did not get 200 response code")
    matchdata_TI19.append(json.loads(matchdata_response.text))
    if fetch_counter % 10 == 0:
        print('Request count: ', fetch_counter)
    fetch_counter +=1
    time.sleep(1.5)


Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Request count:  10
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Request count:  20
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Request count:  30
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  200
Server Response Code:  20

In [38]:
clean_matchdata_TI19 = copy.deepcopy(matchdata_TI19)

team_list_TI19 = []
team_name_list = []

real_arr = []
skill_arr = []
luck_arr = []

# Create list of team names which are competing
for match in matchdata_TI19:
    if 'radiant_team' in match.keys():
        if match['radiant_team']['name'] not in team_name_list:
            team_name_list.append(match['radiant_team']['name'])
    else:
        clean_matchdata_TI19.remove(match)
    if 'dire_team' in match.keys():
        if match['dire_team']['name'] not in team_name_list:
            team_name_list.append(match['dire_team']['name'])
    else:
        if match in clean_matchdata_TI19:
            clean_matchdata_TI19.remove(match)

print('Length raw: ', len(matchdata_TI19))
print('Length cleaned: ', len(clean_matchdata_TI19))

# Create dota teams object from name list        
for name in team_name_list:
    team_list_TI19.append(DotaTeam(name, 0, 0, 0, 0, 0, 0))

# Increment win / loss values from results
for match in clean_matchdata_TI19:
    if match['radiant_win']:
        winner_team = match['radiant_team']['name']
        looser_team = match['dire_team']['name']
    else:
        winner_team = match['dire_team']['name']
        looser_team = match['radiant_team']['name']

    for team in team_list_TI19:
        if team.name == winner_team:
            team.wins +=1
        if team.name == looser_team:
            team.losses +=1
            
# Summary numbers
number_games_per_season = len(clean_matchdata_TI19)
number_games_per_team = len(team_list_TI19)/len(clean_matchdata_TI19)

print('number_games_per_season: ', number_games_per_season)
print('number_games_per_team: ', number_games_per_team)

            
# Calculate winrate and cointoss
for team in team_list_TI19:
    team.winrate = team.wins/(team.wins + team.losses)
    team.real = (number_games_per_season - team.wins) / (len(team_list_TI19)-1)
    team.luck = math.sqrt(0.5*(0.5/number_games_per_season))
    team.skill = team.real - team.luck
    
for team in team_list_TI19:
    real_arr.append(team.real)
    skill_arr.append(team.skill)
    luck_arr.append(team.luck)
    
sd_real = np.std(real_arr)
sd_skill = np.std(skill_arr)
sd_luck = np.std(luck_arr)

contribution_luck = sd_luck / sd_real
contribution_skill = sd_skill / sd_real

print('SD real: ', sd_real)
print('SD skill: ', sd_skill)
print('SD luck: ', sd_luck)
print('Contribution luck: ', contribution_luck)
print('Contribution skill: ', contribution_skill)

Length raw:  461
Length cleaned:  442


In [41]:
team_list_TI19

[{'name': 'BOOM ID', 'wins': 6, 'losses': 8, 'winrate': 0.42857142857142855, 'skill': 3.767521773118362, 'real': 3.791304347826087, 'luck': 0.023782574707724706},
 {'name': 'Mineski', 'wins': 32, 'losses': 22, 'winrate': 0.5925925925925926, 'skill': 3.541434816596623, 'real': 3.5652173913043477, 'luck': 0.023782574707724706},
 {'name': 'Adroit', 'wins': 16, 'losses': 16, 'winrate': 0.5, 'skill': 3.6805652513792317, 'real': 3.7043478260869565, 'luck': 0.023782574707724706},
 {'name': 'Team Jinesbrus', 'wins': 24, 'losses': 14, 'winrate': 0.631578947368421, 'skill': 3.6110000339879273, 'real': 3.634782608695652, 'luck': 0.023782574707724706},
 {'name': 'Team EVOS', 'wins': 4, 'losses': 10, 'winrate': 0.2857142857142857, 'skill': 3.7849130774661885, 'real': 3.8086956521739133, 'luck': 0.023782574707724706},
 {'name': '496 Gaming', 'wins': 6, 'losses': 8, 'winrate': 0.42857142857142855, 'skill': 3.767521773118362, 'real': 3.791304347826087, 'luck': 0.023782574707724706},
 {'name': 'RESURGE